## Unit 13. GBDT 三巨頭 課堂作業
- 課程編號: CHE-AI-114
- 學年度: 114下
- 上課時間: 每週四 09:10-12:00
- 授課地點: 學思樓 705
- 指導教授: 莊曜禎 助理教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 0. 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P3_Unit13_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_3' / 'Unit13'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

# sklearn 套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             accuracy_score, precision_score, recall_score, f1_score,
                             classification_report, confusion_matrix)

# GBDT 三巨頭
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

print("✓ 套件版本:")
print(f"  - XGBoost: {xgb.__version__}")
print(f"  - LightGBM: {lgb.__version__}")
print(f"  - CatBoost: {import('catboost').__version__}")

# 設定隨機種子
SEED = 42
np.random.seed(SEED)

# 設定 matplotlib 中文顯示
plt.rcParams['axes.unicode_minus'] = False

---
### **I. 練習題: 建立 GBDT 三巨頭模型（回歸任務）**

**作業目標**：
1. 使用複雜的化工製程模擬數據集（超大數據量）
2. 建立並訓練 XGBoost、LightGBM、CatBoost 三種模型
3. 比較三種模型的性能表現
4. 分析各模型的優缺點

**案例背景**：
某大型化工廠的蒸餾塔系統，需要預測產品純度。系統包含：
- 多個操作變數（溫度、壓力、流量等）
- 類別變數（設備狀態、操作模式、班次等）
- 時間特徵（小時、星期、月份等）
- 複雜的非線性關係

---
### **1. 數據生成**
生成超大規模化工製程模擬數據（100,000+ 樣本）

In [ ]:
# ========================================
# 生成超大規模化工製程數據
# ========================================

print("開始生成數據...")
start_time = time.time()

# 設定樣本數量（超大數據量）
n_samples = 100000

# ========================================
# 1. 連續型特徵（操作變數）
# ========================================
data = {
    # 蒸餾塔進料條件
    'Feed_Flow': np.random.uniform(800, 1200, n_samples),           # 進料流量 (kg/h)
    'Feed_Temp': np.random.uniform(80, 120, n_samples),             # 進料溫度 (°C)
    'Feed_Pressure': np.random.uniform(2.0, 5.0, n_samples),        # 進料壓力 (bar)
    'Feed_Composition': np.random.uniform(0.3, 0.7, n_samples),     # 進料組成 (mol%)
    
    # 塔頂操作條件
    'Top_Temp': np.random.uniform(60, 90, n_samples),               # 塔頂溫度 (°C)
    'Top_Pressure': np.random.uniform(1.0, 2.5, n_samples),         # 塔頂壓力 (bar)
    'Reflux_Ratio': np.random.uniform(1.5, 4.0, n_samples),         # 回流比
    'Condenser_Duty': np.random.uniform(500, 1500, n_samples),      # 冷凝器負荷 (kW)
    
    # 塔底操作條件
    'Bottom_Temp': np.random.uniform(140, 180, n_samples),          # 塔底溫度 (°C)
    'Bottom_Pressure': np.random.uniform(2.5, 4.0, n_samples),      # 塔底壓力 (bar)
    'Reboiler_Duty': np.random.uniform(1000, 2500, n_samples),      # 再沸器負荷 (kW)
    
    # 冷卻水系統
    'Cooling_Water_Flow': np.random.uniform(50, 150, n_samples),    # 冷卻水流量 (m³/h)
    'Cooling_Water_Temp': np.random.uniform(25, 35, n_samples),     # 冷卻水溫度 (°C)
    
    # 環境條件
    'Ambient_Temp': np.random.uniform(15, 35, n_samples),           # 環境溫度 (°C)
    'Ambient_Humidity': np.random.uniform(40, 80, n_samples),       # 環境濕度 (%)
    'Atmospheric_Pressure': np.random.uniform(0.98, 1.02, n_samples), # 大氣壓力 (atm)
}

# ========================================
# 2. 類別型特徵
# ========================================
data['Equipment_Status'] = np.random.choice(['Normal', 'Maintenance', 'High_Load', 'Low_Load', 'Startup'], 
                                            n_samples, p=[0.70, 0.10, 0.10, 0.05, 0.05])
data['Operation_Mode'] = np.random.choice(['Auto', 'Manual', 'Cascade', 'Override'], 
                                         n_samples, p=[0.75, 0.15, 0.08, 0.02])
data['Shift'] = np.random.choice(['Morning', 'Afternoon', 'Night'], 
                                 n_samples, p=[0.35, 0.35, 0.30])
data['Day_of_Week'] = np.random.choice(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], n_samples)

# ========================================
# 3. 時間特徵
# ========================================
data['Hour'] = np.random.randint(0, 24, n_samples)
data['Month'] = np.random.randint(1, 13, n_samples)
data['Quarter'] = ((np.array(data['Month']) - 1) // 3) + 1

# ========================================
# 4. 衍生特徵（交互作用）
# ========================================
df = pd.DataFrame(data)

# 溫度差
df['Temp_Diff'] = df['Bottom_Temp'] - df['Top_Temp']
df['Feed_Top_Diff'] = df['Feed_Temp'] - df['Top_Temp']

# 壓力降
df['Pressure_Drop'] = df['Bottom_Pressure'] - df['Top_Pressure']

# 能耗相關
df['Total_Duty'] = df['Reboiler_Duty'] + df['Condenser_Duty']
df['Energy_Efficiency'] = df['Reboiler_Duty'] / (df['Feed_Flow'] + 0.1)

# 物理特性
df['Vapor_Velocity'] = df['Top_Pressure'] * df['Top_Temp'] / 300
df['Liquid_Holdup'] = df['Bottom_Pressure'] * df['Reflux_Ratio']

# 冷卻效率
df['Cooling_Efficiency'] = df['Cooling_Water_Flow'] / (df['Ambient_Temp'] + 20)

# ========================================
# 5. 生成目標變數（產品純度）- 複雜非線性關係
# ========================================
# 產品純度受多個因素影響，包括溫度、壓力、回流比、進料條件等
purity = (
    # 主要影響因素
    0.30 * (df['Reflux_Ratio'] - 2.75) * 10 +
    0.25 * (df['Feed_Composition'] - 0.5) * 50 +
    0.20 * np.log(df['Feed_Flow'] / 1000 + 1) * 20 +
    
    # 溫度相關（非線性）
    0.15 * (90 - df['Top_Temp']) +
    0.10 * (df['Bottom_Temp'] - 160) * 0.5 +
    -0.08 * (df['Temp_Diff'] - 90)**2 / 100 +
    
    # 壓力相關
    0.12 * (df['Top_Pressure'] - 1.75) * 5 +
    -0.05 * (df['Pressure_Drop'] - 1.5)**2 +
    
    # 能耗相關
    0.08 * np.log(df['Total_Duty'] / 2000 + 1) * 15 +
    0.06 * (df['Reboiler_Duty'] / df['Feed_Flow']) * 100 +
    
    # 冷卻效率
    0.10 * df['Cooling_Efficiency'] * 3 +
    
    # 設備狀態影響
    np.where(df['Equipment_Status'] == 'Normal', 5, 0) +
    np.where(df['Equipment_Status'] == 'High_Load', -3, 0) +
    np.where(df['Equipment_Status'] == 'Maintenance', -5, 0) +
    np.where(df['Equipment_Status'] == 'Startup', -8, 0) +
    
    # 操作模式影響
    np.where(df['Operation_Mode'] == 'Auto', 3, 0) +
    np.where(df['Operation_Mode'] == 'Manual', -2, 0) +
    np.where(df['Operation_Mode'] == 'Override', -4, 0) +
    
    # 時間因素（季節性）
    2 * np.sin(2 * np.pi * df['Month'] / 12) +
    1.5 * np.cos(2 * np.pi * df['Hour'] / 24) +
    
    # 交互作用
    0.05 * df['Feed_Flow'] * df['Feed_Composition'] / 600 +
    0.04 * df['Reflux_Ratio'] * df['Top_Pressure'] +
    -0.03 * df['Ambient_Temp'] * df['Ambient_Humidity'] / 1000 +
    
    # 添加噪音（模擬真實波動）
    np.random.normal(0, 1.5, n_samples)
)

# 限制純度在合理範圍 (90-99.5%)
purity = np.clip(purity + 92, 90, 99.5)

df['Product_Purity'] = purity

# ========================================
# 6. 保存數據
# ========================================
output_file = DATA_DIR / 'distillation_data_large.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')

elapsed_time = time.time() - start_time
print(f"\n✓ 數據生成完成！")
print(f"  - 檔案位置: {output_file}")
print(f"  - 數據形狀: {df.shape}")
print(f"  - 生成時間: {elapsed_time:.2f} 秒")
print(f"\n數據統計摘要:")
print(f"  - 樣本數量: {len(df):,}")
print(f"  - 特徵數量: {df.shape[1] - 1}")
print(f"  - 連續特徵: {df.select_dtypes(include=[np.number]).shape[1] - 1}")
print(f"  - 類別特徵: {df.select_dtypes(include=['object']).shape[1]}")
print(f"  - 產品純度範圍: [{df['Product_Purity'].min():.2f}, {df['Product_Purity'].max():.2f}]")
print(f"  - 產品純度平均: {df['Product_Purity'].mean():.2f} ± {df['Product_Purity'].std():.2f}")

print(f"\n前 5 筆數據預覽:")
print(df.head())

---
### **2. 數據準備與探索**

**任務**：
1. 載入數據並檢查基本統計資訊
2. 繪製目標變數（產品純度）的分布圖
3. 分析連續特徵與目標變數的相關性（繪製相關性熱圖）
4. 分析類別特徵與目標變數的關係（繪製箱型圖）
5. 數據分割：訓練集70%、驗證集15%、測試集15%
6. **重要**: 對連續特徵進行 StandardScaler 標準化
7. 處理類別特徵（LabelEncoder 或保留原始格式給 CatBoost）

In [ ]:
# 請在此撰寫程式碼
# 提示：
# 1. 使用 pd.read_csv() 載入數據
# 2. 使用 df.describe() 檢查統計資訊
# 3. 使用 plt.hist() 或 sns.histplot() 繪製分布圖
# 4. 使用 df.corr() 和 sns.heatmap() 繪製相關性熱圖
# 5. 使用 sns.boxplot() 分析類別特徵
# 6. 使用 train_test_split() 分割數據（兩次：先 70/30，再 30 分成 15/15）
# 7. 使用 StandardScaler() 標準化連續特徵
# 8. 使用 LabelEncoder() 編碼類別特徵（XGBoost/LightGBM 需要）

---
### **3. 模型一：XGBoost**

#### 3.1 模型建立與訓練

**任務**：
1. 使用 `xgb.XGBRegressor()` 建立 XGBoost 模型
2. 設定以下超參數：
   - `n_estimators`: 迭代次數（建議 500-1000）
   - `learning_rate`: 學習率（建議 0.05-0.1）
   - `max_depth`: 樹深度（建議 6-10）
   - `subsample`: 子樣本比例（建議 0.8）
   - `colsample_bytree`: 特徵子樣本比例（建議 0.8）
   - `random_state`: 隨機種子（42）
3. 使用 `eval_set` 設定驗證集
4. 使用 `early_stopping_rounds` 設定早停（建議 50）
5. 記錄訓練時間

In [ ]:
# 請在此撰寫程式碼
# 提示：
# model_xgb = xgb.XGBRegressor(...)
# start_time = time.time()
# model_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=100)
# training_time_xgb = time.time() - start_time

#### 3.2 模型評估

**任務**：
1. 在訓練集、驗證集、測試集上進行預測
2. 計算評估指標：MAE、RMSE、R²
3. 繪製特徵重要性圖（Top 15 特徵）
4. 繪製預測值 vs 實際值散點圖（Parity Plot）
5. 保存模型

In [ ]:
# 請在此撰寫程式碼
# 提示：
# y_pred_train_xgb = model_xgb.predict(X_train)
# y_pred_val_xgb = model_xgb.predict(X_val)
# y_pred_test_xgb = model_xgb.predict(X_test)
# 計算 MAE, RMSE, R²
# 繪製特徵重要性和 Parity Plot
# model_xgb.save_model(MODEL_DIR / 'xgboost_model.json')

---
### **4. 模型二：LightGBM**

#### 4.1 模型建立與訓練

**任務**：
1. 使用 `lgb.LGBMRegressor()` 建立 LightGBM 模型
2. 設定以下超參數：
   - `n_estimators`: 迭代次數（建議 500-1000）
   - `learning_rate`: 學習率（建議 0.05-0.1）
   - `max_depth`: 樹深度（建議 -1 表示不限制，或 15-31）
   - `num_leaves`: 葉節點數（建議 31-127）
   - `subsample`: 子樣本比例（建議 0.8）
   - `colsample_bytree`: 特徵子樣本比例（建議 0.8）
   - `random_state`: 隨機種子（42）
   - `verbose`: 訓練輸出頻率（建議 100）
3. 使用 `eval_set` 設定驗證集
4. 使用 `callbacks` 設定早停
5. 記錄訓練時間

In [ ]:
# 請在此撰寫程式碼
# 提示：
# model_lgb = lgb.LGBMRegressor(...)
# start_time = time.time()
# model_lgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)])
# training_time_lgb = time.time() - start_time

#### 4.2 模型評估

**任務**：
1. 在訓練集、驗證集、測試集上進行預測
2. 計算評估指標：MAE、RMSE、R²
3. 繪製特徵重要性圖（Top 15 特徵）
4. 繪製預測值 vs 實際值散點圖（Parity Plot）
5. 保存模型

In [ ]:
# 請在此撰寫程式碼
# 提示：
# y_pred_train_lgb = model_lgb.predict(X_train)
# y_pred_val_lgb = model_lgb.predict(X_val)
# y_pred_test_lgb = model_lgb.predict(X_test)
# 計算 MAE, RMSE, R²
# 繪製特徵重要性和 Parity Plot
# model_lgb.booster_.save_model(str(MODEL_DIR / 'lightgbm_model.txt'))

---
### **5. 模型三：CatBoost**

#### 5.1 模型建立與訓練

**任務**：
1. 使用 `CatBoostRegressor()` 建立 CatBoost 模型
2. 設定以下超參數：
   - `iterations`: 迭代次數（建議 500-1000）
   - `learning_rate`: 學習率（建議 0.05-0.1）
   - `depth`: 樹深度（建議 6-10）
   - `cat_features`: 指定類別特徵（重要！）
   - `random_state`: 隨機種子（42）
   - `verbose`: 訓練輸出頻率（建議 100）
   - `early_stopping_rounds`: 早停（建議 50）
3. **重要**：CatBoost 可以直接處理類別特徵，無需 LabelEncoder
4. 使用 `eval_set` 設定驗證集
5. 記錄訓練時間

In [ ]:
# 請在此撰寫程式碼
# 提示：
# cat_features = ['Equipment_Status', 'Operation_Mode', 'Shift', 'Day_of_Week']
# model_cat = CatBoostRegressor(..., cat_features=cat_features)
# start_time = time.time()
# model_cat.fit(X_train, y_train, eval_set=(X_val, y_val))
# training_time_cat = time.time() - start_time

#### 5.2 模型評估

**任務**：
1. 在訓練集、驗證集、測試集上進行預測
2. 計算評估指標：MAE、RMSE、R²
3. 繪製特徵重要性圖（Top 15 特徵）
4. 繪製預測值 vs 實際值散點圖（Parity Plot）
5. 保存模型

In [ ]:
# 請在此撰寫程式碼
# 提示：
# y_pred_train_cat = model_cat.predict(X_train)
# y_pred_val_cat = model_cat.predict(X_val)
# y_pred_test_cat = model_cat.predict(X_test)
# 計算 MAE, RMSE, R²
# 繪製特徵重要性和 Parity Plot
# model_cat.save_model(str(MODEL_DIR / 'catboost_model.cbm'))

---
### **6. 模型綜合比較**

**任務**：
1. 製作比較表格，包含：
   - 模型名稱
   - 訓練時間
   - 訓練集 MAE、RMSE、R²
   - 驗證集 MAE、RMSE、R²
   - 測試集 MAE、RMSE、R²
2. 繪製三個模型的測試集 R² 對比柱狀圖
3. 繪製三個模型的測試集 RMSE 對比柱狀圖
4. 繪製三個模型在同一張圖上的 Parity Plot（使用不同顏色）
5. 分析與討論：
   - 哪個模型表現最好？為什麼？
   - 三個模型的優缺點是什麼？
   - 訓練時間差異如何？
   - CatBoost 處理類別特徵的優勢是否明顯？
   - 在化工應用場景中，你會推薦使用哪個模型？

In [ ]:
# 請在此撰寫程式碼
# 提示：
# 1. 建立 DataFrame 存儲所有模型的評估指標
# results = pd.DataFrame({
#     'Model': ['XGBoost', 'LightGBM', 'CatBoost'],
#     'Training Time (s)': [training_time_xgb, training_time_lgb, training_time_cat],
#     'Test MAE': [...],
#     'Test RMSE': [...],
#     'Test R²': [...]
# })
# 
# 2. 繪製對比圖
# 3. 撰寫分析與討論

---
## **II. 額外加分作業（自由繳交）**

- 學生可自由選擇是否繳交加分作業（下週上課前完成 email 電子檔）
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI、問同學、問學長姊、甚至參考以前別人的作業，但一定要自行 [吸收][執行][整理] 結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容（與他人雷同、原創性低、抄襲比例過高），作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉（提供作業給別人複製的也一樣）
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

---
### **實驗1: 超參數調整實驗**

針對三個模型分別進行超參數調整，比較以下組合：

**XGBoost**：
- 學習率：[0.01, 0.05, 0.1]
- 樹深度：[6, 8, 10]
- 記錄每組參數的測試集 R² 和訓練時間

**LightGBM**：
- 葉節點數：[31, 63, 127]
- 學習率：[0.01, 0.05, 0.1]
- 記錄每組參數的測試集 R² 和訓練時間

**CatBoost**：
- 樹深度：[6, 8, 10]
- 學習率：[0.01, 0.05, 0.1]
- 記錄每組參數的測試集 R² 和訓練時間

**要求**：
1. 製作超參數調整結果表格
2. 找出每個模型的最佳超參數組合
3. 分析超參數對模型性能的影響
4. 比較三個模型對超參數的敏感度

In [ ]:
# 請在此撰寫程式碼



---
### **實驗2: 特徵重要性分析實驗**

**任務**：
1. 比較三個模型的特徵重要性排名
2. 找出所有模型都認為重要的 Top 10 特徵
3. 使用 Top 10 特徵重新訓練三個模型
4. 比較使用全特徵 vs Top 10 特徵的性能差異
5. 分析：
   - 特徵選擇對模型性能的影響
   - 三個模型的特徵重要性評估是否一致？
   - 在化工應用中，特徵選擇如何幫助簡化模型？

In [ ]:
# 請在此撰寫程式碼



---
### **實驗3: 數據規模影響實驗**

**任務**：
1. 使用不同數據規模訓練三個模型：
   - 10,000 樣本（10%）
   - 30,000 樣本（30%）
   - 50,000 樣本（50%）
   - 100,000 樣本（100%）
2. 對每個數據規模，記錄：
   - 訓練時間
   - 測試集 R²
   - 測試集 RMSE
3. 繪製曲線圖：
   - X 軸：數據量
   - Y 軸：R² / RMSE / 訓練時間（三條曲線）
4. 分析：
   - 哪個模型對數據量最敏感？
   - 哪個模型在小數據量時表現最好？
   - 訓練時間與數據量的關係如何？
   - 在化工應用中，如何選擇合適的數據量？

In [ ]:
# 請在此撰寫程式碼



---
### **💭 思考題**

1. **GBDT 三巨頭比較**：
   - XGBoost、LightGBM、CatBoost 的主要差異是什麼？
   - 在什麼情況下應該選擇哪個模型？
   - 為什麼 CatBoost 可以直接處理類別特徵？

2. **類別特徵處理**：
   - LabelEncoder 和 One-Hot Encoding 的差異是什麼？
   - 為什麼 GBDT 模型通常不需要 One-Hot Encoding？
   - CatBoost 的 Ordered Target Statistics 如何解決 Target Leakage 問題？

3. **模型性能與效率**：
   - 為什麼 LightGBM 通常訓練速度最快？
   - 樹深度和葉節點數如何影響模型複雜度？
   - 如何在準確度和訓練時間之間取得平衡？

4. **化工應用考量**：
   - 在實時控制系統中，訓練時間重要嗎？
   - 在批次製程優化中，預測準確度和模型可解釋性哪個更重要？
   - 如果數據中有缺失值，應該如何處理？GBDT 模型如何處理缺失值？

5. **特徵工程**：
   - 衍生特徵（如溫度差、壓力降）為什麼重要？
   - 在化工製程中，哪些物理關係可以轉化為特徵？
   - 特徵標準化對 GBDT 模型重要嗎？為什麼？

6. **模型診斷**：
   - 如何判斷模型是否過擬合？
   - 早停（Early Stopping）的作用是什麼？
   - 如果測試集性能遠差於訓練集，應該如何改進？

7. **實務應用**：
   - 在生產環境中部署模型時需要注意什麼？
   - 如何監控模型性能並決定何時需要重新訓練？
   - 模型的可解釋性（特徵重要性）在化工領域為什麼重要？

### **📝 你的回答**

（請在此撰寫你對思考題的回答）

---
## **想對老師說的話**

（請在此撰寫你想對老師說的話）